- apply CV 
- use logistics regression
- extract female/male errors but train together

Demographic on gender: 

| osteo_predict | 0.0	 | 1.0 | Total
| --- | --- | --- | --- |
| Female | 24551 |	24547 |49098|
| Male | 3400 |	3400| 6800|
| Total |	27951 |	27947	| 55898|

confusion matrix in general
__WITHOUT STRATA __ 

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 1952.5 |  842.6 |
| reality_False | 599.1| 2195.6 |

__WITH STRATA__:
       
| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 3716.3 |  1594.2|
| reality_False | 1136.3| 4174.2 |

result in general:

             precision    recall  f1-score   support

        0.0       0.77      0.70      0.73     27941
        1.0       0.72      0.79      0.75     27941

avg / total       0.74      0.74      0.74     55882


- __accuracy__ on average:  0.742850348301844
- __precision__ on average:  0.7236506884953114
- __recall__ on average:  0.785835335611265
- __f1__ on average:  0.7534621355578518
 
 
confusion matrix of __FEMALE__ 
__WITHOUT STRATA__

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 1712.6 | 742.5 |
| reality_False | 522.6 | 1932.1 |

__WITH STRATA__:

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 3261.8| 1411. |
| reality_False | 996. | 3676.8 |

- precision_F:  0.7660763774719339
- recall_F:  0.6980397192261599
- f1_F:  0.7304772355720782

female:     

            precision    recall  f1-score   support

        0.0       0.77      0.70      0.73     24541
        1.0       0.72      0.79      0.75     24541

avg / total       0.74      0.74      0.74     49082



confusion matrix of __MALE__
__WITHOUT STRATA__

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 240.1 | 99.9|
| reality_False |76.5 |263.5 |

__WITH STRATA__: 

| osteo_predict | pos	 | neg |
| --- | --- | --- |
| reality_True | 459.8 | 187.5|
| reality_False |144.3 |503. |

male:              

                  precision    recall  f1-score   support

        0.0       0.76      0.71      0.73      3400
        1.0       0.73      0.78      0.75      3400

avg / total       0.74      0.74      0.74      6800

- precision_M:  0.7611322628703857
- recall_M:  0.7103352386837634
- f1_M:  0.734856960204571

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_predict, cross_validate
import sklearn.linear_model as linear_model

from sklearn.cross_validation import cross_val_score
# from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import make_scorer

import matplotlib.pyplot as plt

%matplotlib inline

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_osteo = pd.read_csv('osteo_cleaning.csv')
print(data_osteo.shape)  

(55898, 191)


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.crosstab(index=data_osteo["sex"], columns=data_osteo["osteo_predict"], margins = True)

osteo_predict,0.0,1.0,All
sex,,,
F,24551,24547,49098
M,3400,3400,6800
All,27951,27947,55898


In [4]:
pd.crosstab(index=data_osteo["sex"], columns=data_osteo["osteo_predict"], margins = True)/data_osteo.shape[0]

osteo_predict,0.0,1.0,All
sex,,,
F,0.439211,0.439139,0.87835
M,0.060825,0.060825,0.12165
All,0.500036,0.499964,1.00000


In [5]:
# extract needed columns
osteo_extract = data_osteo.filter(['Strata','sex', 'Race_combine', 'Age_combine','osteo_predict', 'Hyponatremia_Prior',
    'Drug_antipsych_prior', 'Drug_Estrogens_prior' 'Drug_Glucocorticoids_prior', 
     'Drug_Nsaids_prior', 'Drug_Opiates_prior', 'Drug_Pp_inhibitors_prior' 'Drug_Progesterone_prior', 
    'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior', 'Drug_Thiazide_prior', 
     'Chronic_Hyponatremia', 'Recent_Hyponatremia', 'Liver_Prior',
     'HeartDisease_Prior', 'PulmDisease_Prior', 'CNS_Disease_Prior', 
     'Malignancy_Prior', 'Alcohol_Prior', 'Tobacco_Prior',
     'Drug_Loop_Diuretic_Prior', 'Sodium_Avg_Ever', 'BMI_impute', 'osteo_predict'
     'Lowest_Recent_Hypo_Cat_edit', 'Median_Recent_Hypo_Cat_edit', 'Calcium_Avg_Ever']) 

osteo_extract['Calcium_Avg_Ever'] = osteo_extract['Calcium_Avg_Ever'].fillna(np.mean(osteo_extract['Calcium_Avg_Ever'])) 
osteo_extract['Sodium_Avg_Ever'] = osteo_extract['Sodium_Avg_Ever'].fillna(np.mean(osteo_extract['Sodium_Avg_Ever'])) 

osteo_extract.shape

(55898, 27)

In [6]:
#extract strata not in pair (maybe the missing pieces were drop during data cleaning)
missing_strata = osteo_extract.groupby('Strata').filter(lambda x: len(x) ==1)
missing_strata

,Strata,sex,Race_combine,Age_combine,osteo_predict,Hyponatremia_Prior,Drug_antipsych_prior,Drug_Nsaids_prior,Drug_Opiates_prior,Drug_Seizure_prior,...,PulmDisease_Prior,CNS_Disease_Prior,Malignancy_Prior,Alcohol_Prior,Tobacco_Prior,Drug_Loop_Diuretic_Prior,Sodium_Avg_Ever,BMI_impute,Median_Recent_Hypo_Cat_edit,Calcium_Avg_Ever
822,506,F,Caucasian,from_70_to_80,1.0,0,0,0,0,0,...,0,0,0,0,1,0,141.000000,0.000000,0.0,9.600000
845,519,F,Other,from_70_to_80,1.0,0,0,0,0,0,...,0,0,0,0,0,0,139.600000,0.000000,0.0,9.720000
9835,5430,F,African American,more_than_80,1.0,0,0,0,0,0,...,0,1,0,0,0,0,143.666667,0.000000,0.0,9.231579
10806,5955,F,African American,from_50_to_70,1.0,0,0,0,0,0,...,0,0,0,0,0,0,142.521739,0.000000,0.0,9.495652
16246,2538,F,Other,from_50_to_70,0.0,0,0,0,0,0,...,0,0,0,0,0,0,130.400000,26.070737,0.0,9.340000
18028,8097,F,Other,from_50_to_70,0.0,0,0,0,0,0,...,0,0,0,1,0,0,140.000000,25.270571,0.0,9.700000
18933,28669,F,African American,more_than_80,0.0,0,0,0,0,0,...,0,1,0,0,0,0,137.000000,0.000000,0.0,9.150000
20222,11085,F,Other,from_70_to_80,1.0,0,0,0,0,0,...,0,0,0,0,0,1,127.000000,0.000000,0.0,9.200000
27404,15038,F,Caucasian,more_than_80,0.0,0,0,0,1,0,...,0,0,0,0,0,1,143.884615,22.968750,0.0,7.229231
30557,16776,F,Caucasian,from_50_to_70,0.0,0,0,1,0,0,...,1,1,0,0,0,0,139.095238,22.176920,0.0,7.370476


In [7]:
osteo_extract_1 = osteo_extract.drop(missing_strata.index, axis = 0)
osteo_extract_1.shape

(55882, 27)

### One-hot encoding

In [8]:
# Copy the original data
data_OHE = osteo_extract_1.copy()

numeric_subset = osteo_extract_1.select_dtypes('number')
categorical_subset = osteo_extract_1.select_dtypes('object')

# One hot encode
categorical_subset = pd.get_dummies(categorical_subset) 

# Join the two dataframes using concat. Make sure to use axis = 1 to perform a column bind
data_OHE = pd.concat([numeric_subset, categorical_subset], axis = 1)

#y variable for OHE
y_OHE = data_OHE['osteo_predict']
print(data_OHE.shape)

(55882, 34)


### apply CV

In [16]:
#1st method: use FOR LOOP, then take the mean of accuracy of 10 folds
fold = KFold(10, shuffle = True, random_state = 1)

accuracy = []
precision = []
recall_accs = []

strata = data_OHE['Strata']
all_preds = np.full(osteo_extract.shape[0], 100)

for i, (train_index, test_index) in enumerate(fold.split(strata)):
    
    train_index_strata = strata.iloc[train_index]
    test_index_strata = strata.iloc[test_index] 
    
    X_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
    X_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)
    
    Xtest_reset_index = X_test.reset_index()['index']
     
    y_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)]['osteo_predict']
    y_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)]['osteo_predict']

    lr = linear_model.LogisticRegression() 
    fit = lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    
    all_preds[Xtest_reset_index] = y_pred

    acc= np.mean(y_pred == y_test) #each individual fold
    accuracy.append(acc) #then add to the accuracy list to calculate avg for total 10 folds later
    
#   record precision for each individual fold
    prec=precision_score(y_test, y_pred) #each individual fold
    precision.append(prec) 
   
    # Calculate the recall score and append it to a list for recall scores
    recall = recall_score(y_test, y_pred)
    recall_accs.append(recall)

In [17]:
#remove extreme values in all_preds, since at beginning when created all_preds, default = 100, but we only have 1 or 0
all_pred = all_preds.copy()
index_del = np.where(all_pred == all_pred.max())
all_pred = np.delete(all_pred, index_del)

print('result in general')
print(classification_report(data_OHE['osteo_predict'], all_pred))

print('\nconfusion matrix on entire dataset')
print(confusion_matrix(data_OHE['osteo_predict'], all_pred)) 

result in general
             precision    recall  f1-score   support

        0.0       0.77      0.70      0.73     27941
        1.0       0.72      0.79      0.75     27941

avg / total       0.74      0.74      0.74     55882


confusion matrix on entire dataset
[[19536  8405]
 [ 5990 21951]]


In [18]:
#results in general
F1_general = 2 * (np.mean(precision) * np.mean(recall_accs)) / (np.mean(precision) + np.mean(recall_accs)) 

print('\n __accuracy__ on average: ', np.mean(accuracy))
print('\n __precision__ on average: ', np.mean(precision))
print('\n __recall__ on average: ', np.mean(recall_accs))
print('\n __f1__ on average: ', (F1_general))

#create a result table
n_cv = list(range(1, 11))
results_table = pd.DataFrame(index = range(len(n_cv),4), columns = ['CV_fold','accuracy at ith fold', 'precision at ith fold','recall at ith fold'])
results_table['CV_fold'] = n_cv

# add results to table
results_table['accuracy at ith fold'] = accuracy
results_table['precision at ith fold'] = precision
results_table['recall at ith fold'] = recall_accs
results_table


 __accuracy__ on average:  0.7428592909791487

 __precision__ on average:  0.7236309359551439

 __recall__ on average:  0.7859104386772755

 __f1__ on average:  0.7534859472870533


,CV_fold,accuracy at ith fold,precision at ith fold,recall at ith fold
0,1,0.741213,0.725969,0.774943
1,2,0.737907,0.716290,0.787879
2,3,0.742788,0.722757,0.787748
3,4,0.744803,0.725109,0.788549
4,5,0.746893,0.725723,0.793785
5,6,0.737804,0.718463,0.782068
6,7,0.746801,0.728445,0.786978
7,8,0.745775,0.728447,0.783703
8,9,0.744369,0.725778,0.785539
9,10,0.740240,0.719328,0.787913


In [19]:
#confusion matrix in general
fold = KFold(10,shuffle=True, random_state =1)
accuracy = []
precision = []
recall_accs = []
TP = []
TN = [] 
FP = []
FN = []
strata = data_OHE['Strata']
for i, (train_index, test_index) in enumerate(fold.split(strata)):
       
    train_index_strata = strata.iloc[train_index]
    test_index_strata = strata.iloc[test_index] 
    
    X_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
    X_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)
     
    y_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)]['osteo_predict']
    y_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)]['osteo_predict']
  
    lr = linear_model.LogisticRegression() 
    fit = lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    TP.append(conf_matrix[0, 0]) 
    TN.append(conf_matrix[1,1])
    FP.append(conf_matrix[1,0])
    FN.append(conf_matrix[0,1])
tp_avg = np.mean(TP)
tn_avg = np.mean(TN)
fp_avg = np.mean(FP)
fn_avg = np.mean(FN)
print('TP: ', np.mean(TP))
print('TN: ', np.mean(TN))
print('FP: ', np.mean(FP))
print('FN: ', np.mean(FN))   

import numpy as np
np.array([[tp_avg, fn_avg], [fp_avg, tn_avg ]]) 

TP:  3716.0
TN:  4174.1
FP:  1136.4
FN:  1594.5


array([[3716. , 1594.5],
       [1136.4, 4174.1]])

In [ ]:
coefs = pd.Series(lr.coef_[0], index=X_train.columns)
print (X_train.columns) 
coefs = coefs.sort_values()
plt.subplot(1,1,1)
coefs.plot(kind="bar")
plt.show()
print (coefs.sort_values(ascending = False))

### calculate confusion matrix based on gender

In [ ]:
#of the last fold
gender_err = pd.crosstab(index=X_test["sex_F"], columns=y_test)
gender_err

In [ ]:
#of the last fold
gender_err = pd.crosstab(index=X_test["sex_F"], columns=y_pred)
gender_err

In [21]:
#1st method: use FOR LOOP, then take the mean of accuracy of 10 folds
fold = KFold(10,shuffle=False, random_state = 1)
strata = data_OHE['Strata']
accuracy = []
precision = []
recall_accs = []
TP_f = []
TN_f = []
FP_f = []
FN_f = []
TP_m = []
TN_m = []
FP_m = []
FN_m = []
for i, (train_index, test_index) in enumerate(fold.split(strata)):
                  
    train_index_strata = strata.iloc[train_index]
    test_index_strata = strata.iloc[test_index] 
    
    X_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
    X_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)
     
    y_train = data_OHE.loc[data_OHE['Strata'].isin(train_index_strata)]['osteo_predict']
    y_test = data_OHE.loc[data_OHE['Strata'].isin(test_index_strata)]['osteo_predict']

 
    lr = linear_model.LogisticRegression() 
    fit = lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    
    #combine X_test, y_test, y_pred into separate df to cal confusion matrix based on gender
    testing_data = pd.concat([X_test, y_test], axis=1)
    testing_data['y_pred'] = y_pred    

    female = testing_data.loc[testing_data['sex_F'] == 1]
    male = testing_data.loc[testing_data['sex_M'] == 1]

    conf_matrix_F = confusion_matrix(female['osteo_predict'], female['y_pred'])
    conf_matrix_M = confusion_matrix(male['osteo_predict'], male['y_pred'])
    

    print('\nconfusion matrix GENERAL at iteration', i, 'th:\n', confusion_matrix(y_test, y_pred))  
    print('\nconfusion matrix for FEMALE: \n', conf_matrix_F)
    print('\nconfusion matrix for MALE: \n', conf_matrix_M)
    
    print('FEMALE: ', classification_report(female['osteo_predict'], female['y_pred']))
    print('\nMALE: ', classification_report(male['osteo_predict'], male['y_pred']))
    
    TP_f.append(conf_matrix_F[0, 0]) 
    TN_f.append(conf_matrix_F[1,1])
    FP_f.append(conf_matrix_F[1,0])
    FN_f.append(conf_matrix_F[0,1])
    
    TP_m.append(conf_matrix_M[0, 0]) 
    TN_m.append(conf_matrix_M[1,1])
    FP_m.append(conf_matrix_M[1,0])
    FN_m.append(conf_matrix_M[0,1])
    
    acc= np.mean(y_pred == y_test) #each individual fold
    accuracy.append(acc)


confusion matrix GENERAL at iteration 0 th:
 [[3702 1618]
 [1160 4160]]

confusion matrix for FEMALE: 
 [[3228 1460]
 [1018 3670]]

confusion matrix for MALE: 
 [[474 158]
 [142 490]]
FEMALE:               precision    recall  f1-score   support

        0.0       0.76      0.69      0.72      4688
        1.0       0.72      0.78      0.75      4688

avg / total       0.74      0.74      0.74      9376


MALE:               precision    recall  f1-score   support

        0.0       0.77      0.75      0.76       632
        1.0       0.76      0.78      0.77       632

avg / total       0.76      0.76      0.76      1264


confusion matrix GENERAL at iteration 1 th:
 [[3772 1565]
 [1179 4158]]

confusion matrix for FEMALE: 
 [[3347 1372]
 [1029 3690]]

confusion matrix for MALE: 
 [[425 193]
 [150 468]]
FEMALE:               precision    recall  f1-score   support

        0.0       0.76      0.71      0.74      4719
        1.0       0.73      0.78      0.75      4719

avg / total  

In [22]:
print('accuracy: ', np.mean(accuracy))
print('\nON AVG: confusion matrix for FEMALE: \n', np.array([[np.mean(TP_f), np.mean(FN_f)], [np.mean(FP_f), np.mean(TN_f)]]))
print('\nON AVG: confusion matrix for MALE: \n', 
      np.array([[np.mean(TP_m), np.mean(FN_m)], [np.mean(FP_m), np.mean(TN_m)]])) 

accuracy:  0.7427044894811965

ON AVG: confusion matrix for FEMALE: 
 [[3262.4 1410.4]
 [ 995.9 3676.9]]

ON AVG: confusion matrix for MALE: 
 [[460.  187.3]
 [144.1 503.2]]


In [ ]:
#remove extreme values in all_preds, since at beginning when created all_preds, default = 100, but we only have 1 or 0
all_pred = all_preds.copy()
index_del = np.where(all_pred == all_pred.max())
all_pred = np.delete(all_pred, index_del)

print('result in general')
print(classification_report(data_OHE['osteo_predict'], all_pred))

print('\nconfusion matrix on entire dataset')
print(confusion_matrix(data_OHE['osteo_predict'], all_pred)) 

In [25]:
data_OHE['all_pred'] = all_pred
female = data_OHE.loc[data_OHE['sex_F'] == 1]
print('female: ', classification_report(female['osteo_predict'], female['all_pred']))

male = data_OHE.loc[data_OHE['sex_M'] == 1]
print('male: ', classification_report(male['osteo_predict'], male['all_pred']))

female:               precision    recall  f1-score   support

        0.0       0.77      0.70      0.73     24541
        1.0       0.72      0.79      0.75     24541

avg / total       0.74      0.74      0.74     49082

male:               precision    recall  f1-score   support

        0.0       0.76      0.71      0.73      3400
        1.0       0.73      0.78      0.75      3400

avg / total       0.74      0.74      0.74      6800



In [26]:
recall_F = np.mean(TP_f)/(np.mean(FN_f) +np.mean(TP_f))
precision_F = np.mean(TP_f)/(np.mean(TP_f) + np.mean(FP_f))
F1_F = 2 * (precision_F * recall_F) / (precision_F + recall_F)

recall_M = np.mean(TP_m)/(np.mean(FN_m) +np.mean(TP_m))
precision_M = np.mean(TP_m)/(np.mean(TP_m) + np.mean(FP_m))
F1_M = 2 * (precision_M * recall_M) / (precision_M + recall_M)


print('precision_F: ', precision_F)
print('recall_F: ', recall_F)
print('f1_F: ', F1_F)
print('\nprecision_M: ', precision_M)
print('recall_M: ', recall_M)
print('f1_M: ', F1_M)

precision_F:  0.7661273278068713
recall_F:  0.6981681218969354
f1_F:  0.7305707023770869

precision_M:  0.7614633338851183
recall_M:  0.7106442144291674
f1_M:  0.7351766022055297


### Random Forest